In [76]:
#%pip install lightfm
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

from lightfm import LightFM
from lightfm.data import Dataset

In [77]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products.pkl')
#interaction_data = interaction_data.sample(1000)
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
encoder = LabelEncoder()
#interaction_data['customer_id_encoded'] = encoder.fit_transform(interaction_data['customer_id'])
#interaction_data['product_id_encoded'] = encoder.fit_transform(interaction_data['main_product_id'])
interaction_data.main_product_id = interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)
display(len(interaction_data.main_product_id.unique()), len(product_data.main_product_id.unique()))
#product_data = product_data[product_data['main_product_id'].isin(interaction_data['main_product_id'])]

display(interaction_data, product_data)

3297

3297

,customer_id,main_product_id,amount
0,1940761,3696391,1
1,1940761,3850720,1
2,1940761,4839681,2
3,1940761,5015362,2
4,1940761,6523930,1
...,...,...,...
585121,11128024,3711313,1
585122,11128039,10224959,1
585123,11128082,4914111,1
585124,11128095,10451987,1


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]"
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]"
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[]
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0]


In [102]:
all_product_ids = pd.concat([interaction_data['main_product_id'], product_data['main_product_id']]).drop_duplicates()
all_product_numbers = product_data['productNumber'].drop_duplicates()
dataset = Dataset()
dataset.fit(users=interaction_data['customer_id'], items=all_product_ids, item_features=all_product_numbers)

(interactions, weights) = dataset.build_interactions(
    (row.customer_id, row.main_product_id, row.amount)
    for row in interaction_data.itertuples()
)



In [103]:
# Ensure that your productNumber is in a list (making it an iterable)
item_features = dataset.build_item_features((
    (row.main_product_id, [row.productNumber])
    for row in product_data.itertuples()))

In [104]:
model = LightFM(loss='bpr', random_state=27)
# model.fit(interactions)
model.fit(interactions, item_features=item_features, num_threads=4)

In [126]:
user_id = 1940761
item_ids = interaction_data[interaction_data.customer_id ==  user_id].main_product_id.tolist()
user_mapping, _, item_mapping, item_mapping2 = dataset.mapping()

# Convert the original user_id to internal user index
internal_user_id = user_mapping[user_id]
print(len(item_ids))
internal_item_ids = [item_mapping[item] for item in item_ids]
print(item_ids)
scores = model.predict(internal_user_id, np.array(internal_item_ids))
print(scores)
# scores = model.predict(user_id, item_ids, item_features=item_features)
top_items = [item_ids[i] for i in scores.argsort()[::-1]]
print(top_items)

6
[3696391, 3850720, 4839681, 5015362, 6523930, 6596395]
[-0.06614877 -0.09934822 -0.14930235 -0.06843994 -0.14963312 -0.15634294]
[3696391, 5015362, 3850720, 4839681, 6523930, 6596395]


In [153]:
def predict_best_n_products(user_id, n = 5):
    item_ids = interaction_data[interaction_data.customer_id ==  user_id].main_product_id.tolist()
    user_mapping, _, item_mapping, _ = dataset.mapping()
    print('user ' + str(user_id) + ' bought following items')
    display(product_data[product_data.main_product_id.isin(item_ids) ])
    
    
    # Convert the original user_id to internal user index
    internal_user_id = user_mapping[user_id]
    
    all_item_ids = np.array(all_product_ids.tolist())
    # Get the internal item indices for these IDs
    internal_all_item_ids = np.array([item_mapping[item] for item in all_item_ids])
    # Convert the list of item_ids that the user has already interacted with to internal item indices
    internal_item_ids = [item_mapping[item] for item in item_ids]
    # Remove the items the user has already interacted with
    recommendable_item_ids = np.setdiff1d(internal_all_item_ids, internal_item_ids)
    # Make predictions for the remaining items
    scores = model.predict(internal_user_id, recommendable_item_ids)
    indices = np.flip(np.argsort(scores)[-n:])
    scores_sorted = [scores[i] for i in indices]
    
    top_recommended_item_internal = [recommendable_item_ids[i] for i in scores.argsort()[::-1]]
    
    top_recommended_items = []
    count = 0
    for internal_id in top_recommended_item_internal:
        top_recommended_items.append((list(item_mapping.keys()) [list(item_mapping.values()).index(internal_id)]))
        count += 1
        if count > n: 
            break
    
    print('user got recommended following ' + str(n) + ' items: ')
    display(product_data[product_data.main_product_id.isin(top_recommended_items)])
    return scores_sorted, top_recommended_items


In [154]:
display(predict_best_n_products(user_id, 5))

user 1940761 bought following items


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
158,3696391,4062862457084,Schnürstiefelette Glattleder schwarz,schwarz,51.791.20,6590677,315587,w,PT,H-W,...,3 cm - 5 cm,44.0,9.5,17.0,17.0,TR,0.0,0.0,"[3696391, 3696389, 3692832, 3696387, 3692891, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
327,3850720,4062862156284,Winterstiefelette Glattleder schwarz,schwarz,51.650.87,6590677,315580,w,PT,H-W,...,5 cm - 8 cm,38.0,5.0,8.0,14.0,TR,0.0,1.0,"[3850720, 3850715, 3850723, 3850718, 4031937, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
924,5015362,4062862987543,PG8001 Sneaker low Rauleder grau,grau,8001.11.03,4997827,3692261,m,CN,F-S,...,3 cm - 5 cm,39.0,6.0,3.0,0.0,Gummi-EVA,0.0,0.0,"[5015362, 5015364, 5015368, 5015372, 5015369, ...","[39.0, 40.0, 40.5, 41.0, 42.0, 42.5, 43.0, 44...."
1056,4839681,4064032990605,PG1018 Slipper Materialmix Leder/Textil grün,grün,1018.13.03,3692257,6406253,m,VN,F-S,...,3 cm - 5 cm,43.0,9.0,9.0,0.0,Gummi-EVA,0.0,0.0,"[4839681, 4839682, 4839680, 4839679, 4839678]","[41.0, 42.0, 42.5, 43.0, 44.0]"
1568,6596395,4064032878477,7302 Hausschuhe Textil rot,rot,73.021.15,6590677,8530128,m,PT,H-W,...,bis 3 cm,44.0,9.5,12.0,0.0,EVA,0.0,0.0,"[6596395, 6596394, 6596391, 6596390]","[39.0, 40.0, 43.0, 44.0]"
1707,6523930,4064032277355,PG6000 Pantolette Textil blau,blau,6000.10.03,3692257,8756145,m,PT,H-W,...,bis 3 cm,46.0,11.0,11.0,0.0,Gummi,0.0,0.0,"[6523930, 6523928, 6523923, 6523931]","[39.0, 44.0, 46.0, 47.0]"


user got recommended following 5 items: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
348,3711496,4062862258544,Plateau Stiefelette Rauleder gelb,gelb,53.730.13,6590677,5014851,w,SK,H-W,...,3 cm - 5 cm,40.0,6.5,11.0,12.0,TR,0.0,0.0,"[3711496, 3711498, 4093641, 4093640, 4093169, ...","[35.0, 35.5, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
838,5015505,4062862720423,PG0138 Slipper Materialmix Leder/Textil blau,blau,0138.13.03,3692257,3692261,m,CN,F-S,...,3 cm - 5 cm,40.0,6.5,4.0,0.0,Gummi-EVA,0.0,0.0,"[5015505, 5015511, 5015508, 5015513, 5015510, ...","[40.0, 41.0, 42.0, 42.5, 43.0, 44.0, 45.0, 46.0]"
1361,6554561,4064032776285,Overknee Lederimitat schwarz,schwarz,72.749.17,6590678,315599,w,PT,H-W,...,3 cm - 5 cm,40.0,6.5,11.0,54.0,TR,0.0,0.0,"[6554561, 6554553, 6554557, 6554558, 6554560, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1801,7593580,4064032120033,Riemchensandale Lackleder blau,blau,82.804.86,6590678,315563,w,PT,F-S,...,bis 3 cm,39.0,6.0,10.0,0.0,TPU,0.0,0.0,"[7593580, 7593587, 7593579, 7593578, 7593577, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1865,7811874,4064032888735,Plateau Sandale Rauleder braun,braun,84.684.14,6590677,315564,w,PT,F-S,...,5 cm - 8 cm,42.0,8.0,14.0,0.0,PU-TPU,1.0,1.0,"[7811874, 7811870, 7811873, 7811869, 7811875, ...","[35.0, 35.5, 36.0, 37.5, 38.5, 39.0, 40.0, 40...."
2107,7701799,4065171020765,Pantolette Textil braun,braun,83.702.88,6590677,315572,w,SK,F-S,...,bis 3 cm,42.0,8.0,10.0,0.0,Gummi,1.0,1.0,"[7701799, 7701796, 7701795, 7701797, 7701794, ...","[36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43...."


([0.013858412, 0.010622009, 0.00874101, 0.006170107, 0.0012614947],
 [7701799, 5015505, 6554561, 7811874, 3711496, 7593580])